# Rapport för laboration

## 1 Introduktion

I denna laboration tillämpas multipel linjär regression med omfattande statistisk analys för att undersöka sambandet mellan bostadspriser och ett antal förklarande variabler. Datamaterialet består av bostadsdata från Kalifornien år 1997 och innehåller information om bland annat medianinkomst, bostadsålder, geografiskt läge samt närhet till kust.

### Statistisk modell

Den statistiska modellen kan skrivas som

$$
Y = X\beta + \varepsilon
$$

där \( Y \) representerar bostadspriset, \( X \) designmatrisen av förklarande variabler, ($\beta$) regressionsparametrarna och ($\varepsilon$) slumptermen.

### Parameterskattning

Parametrarna skattas med minsta-kvadratmetoden (Ordinary Least Squares, OLS), vilket ger estimatorn

$$
\hat{\beta} = (X^T X)^{-1} X^T Y.
$$

Under de klassiska antagandena om linearitet, oberoende feltermer, konstant varians och normalfördelade fel möjliggör modellen statistisk inferens.

### Modellutvärdering

Efter skattningen analyseras modellens förklaringsgrad genom ($\R^2$ ), samt dess övergripande signifikans med ett F-test. Vidare genomförs hypotesprövning för enskilda parametrar med t-test,konfidensintervall beräknas och eventuell multikollinearitet studeras via Pearsons korrelationskoefficient.

Syftet är dels att demonstrera hur klassisk statistisk inferens tillämpas inom regressionsanalys, dels att tydliggöra skillnaden mellan statistisk slutledning och renodlad maskininlärning, där fokus i det senare främst ligger på prediktion snarare än parameterinferens. Samtliga beräkningar implementeras från grunden i Python med hjälp av en egenutvecklad klass, där NumPy används för linjär algebra och SciPy för statistiska fördelningar.

## 2 Method
### Datahantering och explorativ analys
Innan regressionsmodellen estimeras genomförs en inledande datarensning och explorativ analys. Syftet är att identifiera saknade värden, potentiell multikollinearitet samt övergripande beroendestrukturer mellan variablerna.

Variabeln total_bedrooms innehåller saknade observationer. Vid analys av Pearsons korrelationskoefficient konstateras en stark positiv linjär korrelation mellan total_bedrooms och total_rooms. Detta indikerar potentiell multikollinearitet, vilket kan leda till instabila parameterskattningar och förhöjda standardfel. Mot denna bakgrund exkluderas total_bedrooms från den fortsatta analysen.

Vidare observeras höga korrelationer mellan variablerna total_rooms, population och households. För att reducera dimensionsproblematik och minska risken för överanpassning inkluderas enbart total_rooms som representativ variabel.

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd 
import os


path = os.path.join(os.getcwd(),"Data", "housing.csv")
df = pd.read_csv(path)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [4]:
from LinearFunction import LinearRegress
X_overview = df.drop(['ocean_proximity'],axis = 1)
X_overview = X_overview.dropna(axis=0)
X_overview.insert(0,'Intercept',1)

corr_matrix = LinearRegress.corr(X_overview.to_numpy())
col_names = [col for col in X_overview.columns if col!="Intercept"]
corr_df = pd.DataFrame(corr_matrix, index=col_names, columns = col_names)
corr_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.924616,-0.109357,0.045480,0.069608,0.100270,0.056513,-0.015550,-0.045398
latitude,-0.924616,1.000000,0.011899,-0.036667,-0.066983,-0.108997,-0.071774,-0.079626,-0.144638
housing_median_age,-0.109357,0.011899,1.000000,-0.360628,-0.320451,-0.295787,-0.302768,-0.118278,0.106432
total_rooms,0.045480,-0.036667,-0.360628,1.000000,0.930380,0.857281,0.918992,0.197882,0.133294
total_bedrooms,0.069608,-0.066983,-0.320451,0.930380,1.000000,0.877747,0.979728,-0.007723,0.049686
population,0.100270,-0.108997,-0.295787,0.857281,0.877747,1.000000,0.907186,0.005087,-0.025300
households,0.056513,-0.071774,-0.302768,0.918992,0.979728,0.907186,1.000000,0.013434,0.064894
median_income,-0.015550,-0.079626,-0.118278,0.197882,-0.007723,0.005087,0.013434,1.000000,0.688355
median_house_value,-0.045398,-0.144638,0.106432,0.133294,0.049686,-0.025300,0.064894,0.688355,1.000000


### Tolkning av korrelationsmatrisen
Den explorativa korrelationsanalysen används för att identifiera linjära samband och potentiell multikollinearitet mellan variablerna.

En mycket stark negativ korrelation observeras mellan longitude och latitude (−0,92), vilket speglar Kaliforniens geografiska utformning. Ett så starkt samband kan indikera risk för multikollinearitet i en regressionsmodell. Samtidigt utgör dessa variabler ett koordinatsystem för att bestämma bostadens geografiska position. Även om deras enkla korrelation med bostadspriset är relativt svag, är bostadens geografiska läge i praktiken av stor betydelse för prisnivån.

Median_income uppvisar den starkaste positiva korrelationen med bostadspriset (≈ 0.68–0.69), vilket indikerar att inkomst är den viktigaste enskilda prediktorn i det linjära sambandet.

Flera bostadsrelaterade volymvariabler (total_rooms, households, population) visar mycket höga inbördes korrelationer (0.86–0.94). För att minska multikollinearitet inkluderas endast total_rooms i den slutliga modellen.

Variabeln total_bedrooms exkluderas både på grund av saknade värden och stark korrelation med total_rooms.

Housing_median_age uppvisar svaga korrelationer med priset. Alla variablerna slutliga betydelse avgörs dock genom regressionsanalysens t-test, eftersom partiella effekter kan uppstå när övriga variabler kontrolleras för.

In [5]:
X_sample,y_sample = df.drop(['median_house_value','ocean_proximity','total_bedrooms','population','households'],axis = 1), df["median_house_value"]
X_sample.insert(0,'Intercept',1)
model_1 = LinearRegress(X_sample.to_numpy(),y_sample.to_numpy())
model_1.lsm()
model_1.beta


array([-3.77804266e+06, -4.48891674e+04, -4.40287730e+04,  1.15920020e+03,
        3.36448415e+00,  3.72737058e+04])

In [6]:
model_1.n, model_1.d

(20640, 5)

In [7]:
model_1.predict()

array([401749.36241384, 399033.65494204, 378444.40860107, ...,
        18022.94578082,  28577.61958062,  47856.47022009], shape=(20640,))

### Modell 1 – Skattad regressionsfunktion
Modell 1 inkluderar fem förklarande variabler: longitude, latitude, housing_median_age, total_rooms samt median_income. Skattningen baseras på 20 640 observationer och genomförs med minsta-kvadratmetoden (OLS), där parametrarna estimeras enligt $$\hat{\beta} = (X^T X)^{-1} X^T Y.$$

$$\hat{\bf{\beta}} = 
\begin{bmatrix}
-3.77804266e+06 \\
-4.48891674e+04 \\
-4.40287730e+04 \\
1.15920020e+03 \\
3.36448415e+00 \\
3.72737058e+04
\end{bmatrix}$$

Den skattade regressionsfunktionen erhålls som $$\hat{Y} = -3778043 - 44889 x_1 - 44029 x_2 + 1159 x_3 + 3 x_4 + 37274 x_5$$
där
x_1 ​= longitude,x_2 ​= latitude, x_3 ​= housing_median_age, x_4 ​= total_rooms, x_5 ​= median_income.

### Tolkning av koefficienter

Koefficienten för median_income är positiv och relativt stor i jämförelse med övriga variabler, vilket indikerar ett starkt positivt samband mellan inkomstnivå och bostadspris, givet att övriga variabler hålls konstanta.

De negativa koefficienterna för longitude och latitude indikerar att det geografiska läget har en systematisk påverkan på bostadspriserna. Mer specifikt innebär detta att bostäder belägna längre västerut respektive längre söderut – givet övriga variabler konstanta – tenderar att ha högre priser. Denna tolkning är förenlig med den faktiska bostadsmarknadens struktur i Kalifornien.

Den slutliga bedömningen av variablernas statistiska betydelse baseras på t-test och konfidensintervall i den inferentiella analysen.

### Modell 2 – Inkludering av kategorisk variabel

I Modell 2 inkluderas den kategoriska variabeln ocean_proximity för att fånga geografiska lägeseffekter som inte fullt ut representeras av de kontinuerliga koordinaterna longitude och latitude.

Ett envägs-ANOVA F-test genomfördes för att undersöka om ocean_proximity har samband med longtitude och latitude. Resultat uppvisar ocean_proximity ett starkt samband med de geografiska koordinaterna. För att minska risken för multikollinearitet och förbättra modellens tolkbarhet exkluderas därför longitude och latitude i Modell 2.

Den kategoriska variabeln transformeras genom dummy encoding med hjälp av pd.get_dummies(drop_first=True), där en kategori används som referensgrupp för att undvika perfekt multikollinearitet (dummyvariabelfällan).

Den slutliga specifikationen för Modell 2 omfattar:

Förklarande ariabler:housing_median_age, total_rooms, median_income.

Kategoriska variabler (dummy-kodade): ocean_proximity (med en referenskategori).

Parametrarna skattas därefter med minsta-kvadratmetoden enligt samma princip som i Modell 1.

In [8]:
df['lon_lat'] = df['longitude'] * df['latitude']
groups_interaction = [df[df['ocean_proximity'] == cat]['lon_lat'].values 
                      for cat in df['ocean_proximity'].unique()]

f_stat_interaction, p_value_interaction = st.f_oneway(*groups_interaction)
f_stat_interaction, p_value_interaction

(np.float64(2307.7387574101595), np.float64(0.0))

In [9]:
X_dum = pd.get_dummies(df['ocean_proximity'], prefix='ocean',drop_first=True,dtype=float)
feature_cols = ['median_income','housing_median_age','total_rooms']
X_cat = pd.concat([df[feature_cols],X_dum],axis=1)
X_cat.head()

,median_income,housing_median_age,total_rooms,ocean_INLAND,ocean_ISLAND,ocean_NEAR BAY,ocean_NEAR OCEAN
0,8.3252,41.0,880.0,0.0,0.0,1.0,0.0
1,8.3014,21.0,7099.0,0.0,0.0,1.0,0.0
2,7.2574,52.0,1467.0,0.0,0.0,1.0,0.0
3,5.6431,52.0,1274.0,0.0,0.0,1.0,0.0
4,3.8462,52.0,1627.0,0.0,0.0,1.0,0.0


In [10]:
X_cat.insert(0,'Intercept',1)
model_2 = LinearRegress(X_cat.to_numpy(),y_sample.to_numpy())
model_2.lsm()
model_2.beta

array([ 3.85703377e+04,  3.75629245e+04,  1.14362551e+03,  3.46708615e+00,
       -7.14848091e+04,  1.84832226e+05,  1.21015890e+04,  1.78744663e+04])

In [11]:
model_2.predict()

array([413330.46743362, 411125.76851869, 387835.83676161, ...,
        58198.94615397,  64257.06069393,  84762.17321991], shape=(20640,))

### Modell 2 – Skattad regressionsfunktion

Efter dummy-kodning av den kategoriska variabeln ocean_proximity skattas Modell 2 med minsta-kvadratmetoden. Modellen kan skrivas som 
\begin{equation}
    \hat{\bf{Y}} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 t_1 + \beta_5 t_2 + \beta_6 t_3 + \beta_7 t_4
\end{equation}

$$\hat{Y}_t0 = 38570 + 37563 x_1 + 1144 x_2 + 3 x_3$$

$$\hat{Y}_t1 = -32914 + 37563 x_1 + 1144 x_2 + 3 x_3$$

$$\hat{Y}_t2 = 223403 + 37563 x_1 + 1144 x_2 + 3 x_3$$

$$\hat{Y}_t3 = 50672 + 37563 x_1 + 1144 x_2 + 3 x_3$$

$$\hat{Y}_t4 = 56445 + 37563 x_1 + 1144 x_2 + 3 x_3$$

där x1​=median_income,x2​=housing_median_age, x3​=total_rooms, , och $t_1, t_2, t_3, t_4 $ är dummyvariabler för ocean_proximity, med kategorin <1H OCEAN som referensgrupp. De skattade parametrarna visar tydliga geografiska skillnader i prisnivå.


### Tolkning av dummyvariabler
Dummykoefficienterna anger den förväntade skillnaden i bostadspris relativt referenskategorin, givet att de kontinuerliga variablerna hålls konstanta.

INLAND: −71 485
→ Signifikant lägre pris än referenskategorin.

ISLAND: +184 832
→ Avsevärt högre pris än referenskategorin.

NEAR BAY: +12 102
→ Måttligt högre pris.

NEAR OCEAN: +17 874
→ Något högre pris.

Resultaten är ekonomiskt intuitiva: närhet till kust eller ö-läge är förknippat med högre bostadspriser, medan inlandslägen har lägre prisnivå, allt annat lika.

Genom att ersätta longitude och latitude med den kategoriska variabeln ocean_proximity förändras koefficienterna för de övriga förklarande variablerna endast marginellt. Detta indikerar att Modell 2 i huvudsak fångar samma geografiska information som Modell 1.

Skillnaden ligger främst i hur lägeseffekten specificeras. I stället för en kontinuerlig geografisk gradient (koordinater) använder Modell 2 en kategorisk indelning, vilket ger en mer direkt och tolkbar representation av kustnära respektive inlandslägen.

In [12]:
model1_stats = pd.Series({'Variance (σ²)': model_1.var(),'Std Dev (σ)': model_1.std(),'RMSE': model_1.rmse(),'R²': model_1.R2(),'F-statistic': model_1.f_test()[0],'F-test p-value': model_1.f_test()[1]}, name='Model 1')
model2_stats = pd.Series({'Variance (σ²)': model_2.var(),'Std Dev (σ)': model_2.std(),'RMSE': model_2.rmse(),'R²': model_2.R2(),'F-statistic': model_2.f_test()[0],'F-test p-value': model_2.f_test()[1]}, name='Model 2')
comparison_df = pd.DataFrame([model1_stats, model2_stats]).T

comparison_df

,Model 1,Model 2
Variance (σ²),5.361843e+09,5.321353e+09
Std Dev (σ),7.322461e+04,7.294760e+04
RMSE,7.321396e+04,7.293346e+04
R²,5.974404e-01,6.005190e-01
F-statistic,6.124600e+03,4.430717e+03
F-test p-value,0.000000e+00,0.000000e+00


### Varians och standardavvikelse

Den väntevärdesriktiga felvariansen skattas enligt

$$
\hat{\sigma}^2 = \frac{SSE}{n - d - 1}
$$

**Modell 1:** 5.361843e+09	

**Modell 2:** 5.321353e+09

Skillnaden är marginell, men Modell 2 uppvisar något lägre residualvarians, vilket indikerar en svagt förbättrad anpassning.

**Standardavvikelsen** $(\sqrt{\hat{\sigma}^2})$ uppgår till:

- Modell 1: 73 225 USD  
- Modell 2: 72 949 USD

Detta innebär att prediktionerna i genomsnitt avviker cirka 73 000 USD från observerade priser. Givet ett genomsnittligt bostadspris om cirka 207 000 USD motsvarar detta en relativ avvikelse på cirka 35 %.

Skillnaden mellan modellerna är praktiskt försumbar.

### Förklaringsgrad (R²)

- **Modell 1:** 0,597  
- **Modell 2:** 0,601

Modell 2 förklarar 60,1 % av variationen i bostadspriser, vilket är en marginell ökning om 0,4 procentenheter.

Detta indikerar att *ocean_proximity* fångar ungefär samma geografiska information som *longitude* och *latitude*, men i en mer strukturerad form.

### F-test (samlad modellprövning)

Modell 1: F = 6 125

Modell 2: F = 4 431

p < 0,001 i båda modeller

Nollhypotesen att samtliga koefficienter (utom interceptet) är lika med noll förkastas med mycket stark statistisk säkerhet.

Det lägre F-värdet i Modell 2 beror på fler parametrar (dummyvariabler) och innebär inte svagare samband.

Slutsats: Båda modellerna är globalt signifikanta.

In [13]:
t1, p1 = model_1.t_test()
featuresmodel1 = ['Intercept', 'longitude', 'latitude', 'housing_median_age', 'total_rooms', 'median_income']

model1_ttest = pd.DataFrame({'Feature': featuresmodel1,'Coefficient': model_1.beta,'t-value': t1,'p-value': p1,'Significant (α=0.05)': p1 < 0.05})

model1_ttest['t-value'] = model1_ttest['t-value'].apply(lambda x: f'{x:.4f}')
model1_ttest['p-value'] = model1_ttest['p-value'].apply(lambda x: f'{x:.4e}' if x < 0.0001 else f'{x:.4f}')
model1_ttest['Coefficient'] = model1_ttest['Coefficient'].apply(lambda x: f'{x:.4f}')
model1_ttest

,Feature,Coefficient,t-value,p-value,Significant (α=0.05)
0,Intercept,-3778042.6604,-59.4441,0.0000e+00,True
1,longitude,-44889.1674,-62.3886,0.0000e+00,True
2,latitude,-44028.7730,-65.4282,0.0000e+00,True
3,housing_median_age,1159.2002,25.5382,1.1743e-141,True
4,total_rooms,3.3645,13.2309,8.4480e-40,True
5,median_income,37273.7058,130.6303,0.0000e+00,True


In [14]:
t2, p2 = model_2.t_test()
featuresmodel2 = ['Intercept', 'median_income', 'housing_median_age', 'total_rooms', 'ocean_INLAND', 'ocean_ISLAND', 'ocean_NEAR BAY', 'ocean_NEAR OCEAN']
model2_ttest = pd.DataFrame({'Feature': featuresmodel2,'Coefficient': model_2.beta,'t-value': t2,'p-value': p2,'Significant (α=0.05)': p2 < 0.05})

# Formatera för läsbarhet
model2_ttest['t-value'] = model2_ttest['t-value'].apply(lambda x: f'{x:.4f}')
model2_ttest['p-value'] = model2_ttest['p-value'].apply(lambda x: f'{x:.4e}' if x < 0.0001 else f'{x:.4f}')
model2_ttest['Coefficient'] = model2_ttest['Coefficient'].apply(lambda x: f'{x:.4f}')
model2_ttest

,Feature,Coefficient,t-value,p-value,Significant (α=0.05)
0,Intercept,38570.3377,17.1433,2.0029e-65,True
1,median_income,37562.9245,132.3513,0.0000e+00,True
2,housing_median_age,1143.6255,24.8176,5.3217e-134,True
3,total_rooms,3.4671,13.6663,2.4670e-42,True
4,ocean_INLAND,-71484.8091,-57.4647,0.0000e+00,True
5,ocean_ISLAND,184832.2257,5.6630,1.5069e-08,True
6,ocean_NEAR BAY,12101.5890,6.9312,4.2961e-12,True
7,ocean_NEAR OCEAN,17874.4663,11.1101,1.3504e-28,True


### Tolkning av t-värden

**t-värdet** definieras som

$$
t = \frac{\hat{\beta}_i}{\text{SE}(\hat{\beta}_i)}
$$

Samtliga variabler i båda modellerna uppvisar \(|t| \gg 2\), vilket innebär signifikans långt under 5 %-nivån.

### Viktiga observationer:

- **median_income** har extremt högt t-värde i Modell 1 (≈130), vilket bekräftar att inkomst är den dominerande prediktorn.
- I Modell 2 minskar vissa t-värden, vilket delvis beror på ändrad skala och omfördelning av förklarad variation.
- **housing_median_age** och **total_rooms** har lägre effektstorlek men fortfarande mycket höga t-värden (över 10), vilket innebär att deras skattningar är statistiskt mycket väl säkerställda.

Det centrala är att **ingen variabel förlorar signifikans** mellan modellerna.


In [15]:
feature_names1= [col for col in featuresmodel1]
lower99_1,upper99_1 = model_1.interval(0.01)
lower95_1,upper95_1 = model_1.interval()
lower90_1,upper90_1 = model_1.interval(0.10)

interval_df1 = pd.DataFrame({'Feature': feature_names1, 'Coefficient':model_1.beta, 'CI_90_Lower': lower90_1, 'CI_90_Upper': upper90_1,'CI_95_Lower': lower95_1,'CI_95_Upper': upper95_1,'CI_99_Lower': lower99_1,'CI_99_Upper': upper99_1}).round(2)
interval_df1 

,Feature,Coefficient,CI_90_Lower,CI_90_Upper,CI_95_Lower,CI_95_Upper,CI_99_Lower,CI_99_Upper
0,Intercept,-3778042.66,-3882588.08,-3673497.24,-3902617.93,-3653467.39,-3941767.86,-3614317.46
1,longitude,-44889.17,-46072.71,-43705.63,-46299.46,-43478.87,-46742.67,-43035.66
2,latitude,-44028.77,-45135.70,-42921.85,-45347.77,-42709.77,-45762.29,-42295.25
3,housing_median_age,1159.20,1084.54,1233.86,1070.23,1248.17,1042.27,1276.13
4,total_rooms,3.36,2.95,3.78,2.87,3.86,2.71,4.02
5,median_income,37273.71,36804.35,37743.07,36714.42,37832.99,36538.66,38008.75


In [16]:
feature_names2= [col for col in featuresmodel2]
lower99,upper99 = model_2.interval(0.01)
lower95,upper95 = model_2.interval()
lower90,upper90 = model_2.interval(0.10)

interval_df2 = pd.DataFrame({'Feature': feature_names2, 'Coefficient':model_2.beta, 'CI_90_Lower': lower90, 'CI_90_Upper': upper90,'CI_95_Lower': lower95,'CI_95_Upper': upper95,'CI_99_Lower': lower99,'CI_99_Upper': upper99}).round(2)
interval_df2 

,Feature,Coefficient,CI_90_Lower,CI_90_Upper,CI_95_Lower,CI_95_Upper,CI_99_Lower,CI_99_Upper
0,Intercept,38570.34,34869.45,42271.22,34160.40,42980.28,32774.50,44366.17
1,median_income,37562.92,37096.07,38029.78,37006.63,38119.22,36831.80,38294.04
2,housing_median_age,1143.63,1067.83,1219.43,1053.30,1233.95,1024.92,1262.33
3,total_rooms,3.47,3.05,3.88,2.97,3.96,2.81,4.12
4,ocean_INLAND,-71484.81,-73531.06,-69438.56,-73923.10,-69046.51,-74689.38,-68280.24
5,ocean_ISLAND,184832.23,131144.57,238519.88,120858.56,248805.89,100753.73,268910.72
6,ocean_NEAR BAY,12101.59,9229.61,14973.57,8679.37,15523.81,7603.88,16599.30
7,ocean_NEAR OCEAN,17874.47,15228.03,20520.90,14721.00,21027.93,13729.97,22018.97


### Konfidensintervall

Samtliga 95 %-intervall i båda modellerna exkluderar noll, vilket bekräftar signifikansresultaten från t-testen.

Modell 2 ger dock:

Ett tolkningsbart intercept (baspris i referenskategorin <1H OCEAN)

Direkta och kommunikativa intervall för geografiska prisskillnader

Exempel:

INLAND:[−73923,−69047]

Detta innebär att bostäder i inlandet med 95 % säkerhet är mellan cirka 69 000 och 74 000 USD billigare än jämförbara bostäder nära kusten.

För ocean_ISLAND är intervallet brett, vilket sannolikt beror på få observationer. Detta speglar ökad osäkerhet snarare än modellbrist.

## 3 Resultat

### Modell 1 – Multipel linjär regression med geografiska koordinater

Modell 1 skattades med *longitude*, *latitude*, *housing_median_age*, *total_rooms* och *median_income* som förklarande variabler. Samtliga koefficienter var statistiskt signifikanta på 0,1 %-nivån (p < 0,001).

Modellen uppvisade en förklaringsgrad om \($R^2$ = 0,597\), vilket innebär att 59,7 % av variationen i bostadspriserna kan förklaras av de inkluderade variablerna. RMSE uppgick till 73 214 USD.

Det globala F-testet gav \(F = 6\,125\) (p < 0,001), vilket innebär att nollhypotesen om att samtliga regressionskoefficienter (förutom interceptet) är lika med noll kan förkastas.

*median_income* uppvisade det högsta |t|-värdet (130,6) och är således den mest precisa och inflytelserika prediktorn i modellen. De negativa koefficienterna för *longitude* och *latitude* indikerar att bostäder belägna längre västerut respektive söderut är förknippade med högre priser, givet övriga variabler konstanta.

---

### Modell 2 – Inkludering av kategorisk variabel (*ocean_proximity*)

I Modell 2 ersattes *longitude* och *latitude* med den kategoriska variabeln *ocean_proximity*, vilken transformerades till dummyvariabler med **\<1H OCEAN** som referenskategori. Modellen inkluderade även *housing_median_age*, *total_rooms* och *median_income*.

Samtliga variabler var statistiskt signifikanta (p < 0,001). Förklaringsgraden uppgick till \($R^2$= 0,601\), vilket innebär en marginell ökning om 0,4 procentenheter jämfört med Modell 1. RMSE minskade något till 72 933 USD.

Det globala F-testet gav \(F = 4\,431\) (p < 0,001), vilket bekräftar att modellen är sammantaget signifikant. Det lägre F-värdet jämfört med Modell 1 förklaras av ett större antal parametrar.

Dummyvariablerna visar tydliga geografiska priseffekter:
- *ocean_INLAND* uppvisar en negativ effekt om **−71 485 USD** relativt referenskategorin.
- *ocean_ISLAND* uppvisar den största positiva effekten (**+184 832 USD**), dock med ett relativt brett konfidensintervall, vilket indikerar högre skattningsosäkerhet.
- Kategorierna *NEAR BAY* och *NEAR OCEAN* uppvisar måttligt positiva effekter.

---

### Jämförelse mellan modellerna

| Mått        | Modell 1    | Modell 2    | Skillnad   |
|-------------|-------------|-------------|------------|
| R²          | 0,597       | 0,601       | +0,004     |
| RMSE        | 73 214 USD  | 72 933 USD  | −281 USD   |
| F-värde     | 6 125       | 4 431       | —          |
| p-värde (F) | < 0,001     | < 0,001     | Oförändrat |

Modell 2 uppvisar en marginell förbättring i prediktiv anpassning. Skillnaden i R² är dock mycket liten och RMSE-minskningen om 281 USD motsvarar mindre än 0,4 %, vilket saknar praktisk betydelse.

De två modellerna har därmed jämförbar prediktiv prestanda.

---

## 4 Diskussion

### Tolkning av resultaten

Båda modellerna uppvisar likvärdig förklaringsgrad och prediktionsprecision. Detta är väntat, eftersom *longitude*/*latitude* och *ocean_proximity* representerar samma underliggande geografiska dimension – bostadens läge i förhållande till kusten.

Den marginella ökningen i R² när koordinater ersätts med kategorisk kustnärhet indikerar att dummyvariablerna fångar den relevanta lägeseffekten på ett strukturerat och effektivt sätt.

---

### Modell 2:s främsta styrka

Modell 2:s huvudsakliga fördel ligger inte i förbättrad prediktionsförmåga, utan i ökad tolkningsbarhet. 


Den geografiska effekten uttrycks nu som diskreta och direkt tolkbara prisskillnader mellan kategorier, snarare än som kontinuerliga gradienter i longitud och latitud.

Interceptet i Modell 2 representerar dessutom ett meningsfullt baspris för referenskategorin, vilket underlättar ekonomisk tolkning.

Detta gör modellen mer kommunikativ och praktiskt användbar i besluts- och policykontexter.

---

### Osäkerhet och precision

Det breda konfidensintervallet för *ocean_ISLAND* speglar en högre skattningsosäkerhet, sannolikt till följd av ett litet antal observationer i denna kategori.

Detta är metodologiskt korrekt och visar att modellen explicit redovisar osäkerhet. I Modell 1 absorberas denna effekt implicit genom de kontinuerliga koordinaterna, vilket kan ge en mer homogen men mindre transparent precision.

---

### Begränsningar och vidare analys

Trots att modellerna förklarar cirka 60 % av prisvariationen återstår 40 % oförklarad variation. Detta indikerar att ytterligare relevanta variabler saknas, exempelvis:

- Bostadens standard och skick
- Närmiljöfaktorer
- Tidsvariationer
- Icke-linjära samband

Vidare analys skulle kunna inkludera interaktionseffekter samt undersöka potentiell heteroskedasticitet.

---

### Sammanfattande slutsats

**Modell 2 är att föredra**, inte på grund av överlägsen prediktionsprecision, utan på grund av:

| Aspekt                     | Fördel med Modell 2                        |
|----------------------------|---------------------------------------------|
| Tolkningsbarhet            | ✅ Direkt tolkning av kategoriala effekter  |
| Inferens                   | ✅ Tydligare och mer transparent            |
| Kommunikativ användbarhet  | ✅ Bättre lämpad för policy och beslut      |

Studien illustrerar att modellval inom statistisk inferens bör baseras på teoretisk rimlighet och tolkningsbarhet** snarare än marginella förbättringar i anpassningsmått.

I en inferensorienterad analys är Modell 2 det mer ändamålsenliga valet.